# Analytical solution for 1D unconfined flow with two defined head boundaries
This notebook provide the mathematical solution for 1D groundwater flow in an unconfined aquifer bounded by two defined head boundary conditions.

## Conceptual model

The conceptual model consider the aquifer as homogeneous and isotropic structure with horizontal bottom. The aquifer is bounded by two defined-head boundary conditions on the in- and outflow part. From the top, the aquifer receives uniform groundwater recharge. Figure 1 illustrates the conceptual model.

<br>
<figure>
  <img src="FIGS/concept_1D_flow_unconfined.png" alt="Conceptual model for 1D flow in an unconfined aquifer" style="width:50%">
  <figcaption>Fig.1 - Conceptual model for 1D flow in an unconfined aquifer.</figcaption>
</figure>

## Equations

The equation for 1D groundwater flow in a homogeneous aquifer is

$ \frac{d}{dx}=(-hK\frac{dh}{dx})=R$
with
- _x_ is spatial coordinate along flow,
- _h_ is hydraulic head,
- _K_ is hydraulic conductivity,
- _R_ is recharge.

A solution for the equation can be obtained with two boundary conditions at _x_ = 0 and _x_ = _L_:

- $h(0) = h_0$

- $h(L) = h_L$

The solution for hydraulic head _h_ along _x_ is

$ h(x)=\sqrt{h_0^2-\frac{h_0^2-h_L^2}{L}x+\frac{R}{K}x(L-x)}$

## Computation and visualization

Subsequently, the solution is computed and results are visualized. You can modify the parameters to investigate the functional behavior. You can modify the groundwater recharge _R_ (in mm/a) and the hydraulic conductivity _K_ (in m/s). 

In [3]:
# Initialize librarys
from scipy.special import erfc, erf
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
from ipywidgets import *

# Definition of the function
def head(R, K, hl, hr, L, y_scale):
    x = np.arange(0, L,L/1000)
    R=R/1000/365.25/86400
    h=(hl**2-(hl**2-hr**2)/L*x+(R/K*x*(L-x)))**0.5
    
    # PLOT FIGURE
    fig = plt.figure(figsize=(9,6))
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(x,h)
    ax.set(xlabel='x', ylabel='head',title='Hydraulic head for 1D unconfined flow')
    ax.fill_between(x,0,h, facecolor='lightblue')
    
    # BOUNDARY CONDITIONS hl, hr
    ax.vlines(0, 0, hl, linewidth = 10, color='b')
    ax.vlines(L, 0, hr, linewidth = 10, color='b')
    
    # MAKE 'WATER'-TRIANGLE
    h_arrow = (hl**2-(hl**2-hr**2)/L*(L*0.96)+(R/K*(L*0.96)*(L-(L*0.96))))**0.5  #water level at arrow
    ax.arrow(L*0.96,(h_arrow+(h_arrow*0.002)), 0, -0.01, fc="k", ec="k", head_width=(L*0.015), head_length=(h_arrow*0.0015))
    ax.hlines(y= h_arrow-(h_arrow*0.0005), xmin=L*0.95, xmax=L*0.97, colors='blue')   
    ax.hlines(y= h_arrow-(h_arrow*0.001), xmin=L*0.955, xmax=L*0.965, colors='blue')

    #ARROWS FOR RECHARGE 
    if R != 0:
        head_length=(R*86400*365.25*1000*0.002)*y_scale/2
        h_rch1 = (hl**2-(hl**2-hr**2)/L*(L*0.25)+(R/K*(L*0.25)*(L-(L*0.25))))**0.5  #water level at arrow for Recharge Posotion 1
        ax.arrow(L*0.25,(h_rch1+head_length), 0, -0.01, fc="k", ec="k", head_width=(head_length*300/y_scale), head_length=head_length)
        h_rch2 = (hl**2-(hl**2-hr**2)/L*(L*0.50)+(R/K*(L*0.50)*(L-(L*0.50))))**0.5  #water level at arrow for Recharge Postition 2
        ax.arrow(L*0.50,(h_rch2+head_length), 0, -0.01, fc="k", ec="k", head_width=(head_length*300/y_scale), head_length=head_length)
        h_rch3 = (hl**2-(hl**2-hr**2)/L*(L*0.75)+(R/K*(L*0.75)*(L-(L*0.75))))**0.5  #water level at arrow for Recharge Position 3
        ax.arrow(L*0.75,(h_rch3+head_length), 0, -0.01, fc="k", ec="k", head_width=(head_length*300/y_scale), head_length=head_length)

    #Groundwater divide
    max_y = max(h)
    max_x = x[h.argmax()]
    R_min_ms=K*abs(hl**2-hr**2)/L**2
    if R>R_min_ms:
        plt.vlines(max_x,0,max_y, color="r")

    plt.ylim(hl*(1-y_scale/100),hr*(1+y_scale/100))
    plt.xlim(-50,L+50)
    plt.text(L, (hr*1.016), 'R: {:.2e} m/s '.format(R), horizontalalignment='right', bbox=dict(boxstyle="square", facecolor='grey'))
    ax.grid()
    plt.show()
    #print('R: ',R, ' m/s')

# Computation

interact(head,
         y_scale = widgets.BoundedFloatText(value=3, min=1, max=100, step=1, description='Scal. head:', disabled=False),
         hl=widgets.BoundedFloatText(value=150, min=0, max=1000, step=1, description="$h_0$:", disabled=False),
         hr=widgets.BoundedFloatText(value=152, min=0, max=1000, step=1, description='$h_L$:', disabled=False),
         L= widgets.BoundedFloatText(value=2500,min=0, max=20000,step=100, description='$L$:' , disabled=False),
         R=(-500,500,10),
         K=widgets.FloatLogSlider(value=0.0001,base=10,min=-6, max=-2, step=0.1,readout=True,readout_format='.2e'))

interactive(children=(IntSlider(value=0, description='R', max=500, min=-500, step=10), FloatLogSlider(value=0.…

<function __main__.head(R, K, hl, hr, L, y_scale)>

<hr>
&copy; 2023 | Thomas Reimann
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>